# Intel® Extension for Scikit-learn ElasticNet for Airlines DepDelay dataset

In [1]:
from timeit import default_timer as timer
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

### Download the data

In [2]:
x, y = fetch_openml(name='Airlines_DepDelay_10M', return_X_y=True)
for col in ['UniqueCarrier', 'Origin', 'Dest']:
    le = LabelEncoder().fit(x[col])
    x[col] = le.transform(x[col])

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((9000000, 9), (1000000, 9), (9000000,), (1000000,))

### Normalize the data

In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler_x = MinMaxScaler()
scaler_y = StandardScaler()

In [5]:
scaler_x.fit(x_train)
x_train = scaler_x.transform(x_train)
x_test = scaler_x.transform(x_test)

scaler_y.fit(y_train.to_numpy().reshape(-1, 1))
y_train = scaler_y.transform(y_train.to_numpy().reshape(-1, 1)).ravel()
y_test = scaler_y.transform(y_test.to_numpy().reshape(-1, 1)).ravel()

### Patch original scikit-learn with Intel® Extension for Scikit-learn
Intel® Extension for Scikit-learn (previously known as daal4py) contains drop-in replacement functionality for the stock scikit-learn package. You can take advantage of the performance optimizations of Intel Extension for Scikit-learn by adding just two lines of code before the usual scikit-learn imports:

In [6]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Intel® Extension for Scikit-learn patching affects performance of specific Scikit-learn functionality. Refer to the [list of supported algorithms and parameters](https://intel.github.io/scikit-learn-intelex/algorithms.html) for details. In cases when unsupported parameters are used, the package fallbacks into original Scikit-learn. If the patching does not cover your scenarios, [submit an issue on GitHub](https://github.com/intel/scikit-learn-intelex/issues).

Training of the ElasticNet algorithm with Intel® Extension for Scikit-learn for Airlines DepDelay dataset

In [7]:
from sklearn.linear_model import ElasticNet
params = {
    "alpha": 0.1,    
    "fit_intercept": False,
    "l1_ratio": 0.3,
    "random_state": 0,
    "copy_X": False,
}

In [8]:
start = timer()
model = ElasticNet(**params).fit(x_train, y_train)
f"Intel(R) extension for Scikit-learn time: {(timer() - start):.2f} s"

'Intel(R) extension for Scikit-learn time: 0.06 s'

Predict and get a result of the ElasticNet algorithm with Intel® Extension for Scikit-learn

In [9]:
y_predict = model.predict(x_test)
mse_metric_p = metrics.mean_squared_error(y_test, y_predict)
mse_metric_p

1.018687043426095

### Train the same algorithm with original scikit-learn
In order to cancel optimizations, we use *unpatch_sklearn* and reimport the class ElasticNet

In [10]:
from sklearnex import unpatch_sklearn
unpatch_sklearn()

Training of the ElasticNet algorithm with original scikit-learn library for Airlines DepDelay dataset

In [11]:
from sklearn.linear_model import ElasticNet

In [12]:
start = timer()
model = ElasticNet(**params).fit(x_train, y_train)
f"Original Scikit-learn time: {(timer() - start):.2f} s"

'Original Scikit-learn time: 0.48 s'

Predict and get a result of the ElasticNet algorithm with original Scikit-learn

In [13]:
y_predict = model.predict(x_test)
mse_metric_o = metrics.mean_squared_error(y_test, y_predict)
mse_metric_o

1.018687043426095

### Compare MSE metric of patched scikit-learn and original

In [14]:
print(f"mse metric of unpatched scikit-learn: {mse_metric_o}")
print(f"mse metric of patched scikit-learn: {mse_metric_p}")
print(f"attitude: {mse_metric_p/mse_metric_o}")

mse metric of unpatched scikit-learn: 1.018687043426095
mse metric of patched scikit-learn: 1.018687043426095
attitude: 1.0


### With scikit-learn-intelex patching you can:

- Use your scikit-learn code for training and prediction with minimal changes (a couple of lines of code);
- Fast execution training and prediction of scikit-learn models;
- Get the same quality;
- Get speedup more than **12** times.